### Scaling - Cedric Liang

#### Image Selection

Now that our science images have been processed according to the procedure described in the opening chapters of Rieke, we must process the images further to remove problematic systemic issues in our data.

A great deal of our images were taken in imperfect conditions without the usage of the auto-tracker. As such, there is noticeable movement of the open cluster in our images, and we must computationally correct for these imperfections.

However, it first makes sense to get a statistical overview of the images and use some common sense to discard the ones that are clearly unfit for use. There was intermittent cloud cover on the night of our observations, and as such a reasonable portion (30-40%) of the images were taken such that the background was significantly elevated due to the reflection of terrestrial light - in some cases up to 20,000 counts.

As such, it makes sense for us to first analyse our data and determine which images to simply discard.

There is also a human element to this - it is more difficult for computational methods to discard qualitatively poor images that can impact our results, such as images with poor focus or streaking. Indeed, we noticed on the night of observation that some images had trails. With the quantity of data that we have, it makes more sense to discard these manually.


In [31]:
import numpy as np
import astropy
import ccdproc
from ccdproc import CCDData, Combiner
from astropy import units as u
from astropy.visualization import SqrtStretch
import matplotlib.pyplot as plt 
from matplotlib.colors import LogNorm
from photutils.centroids import centroid_com, centroid_1dg, centroid_2dg
from photutils import CircularAperture
from photutils import aperture_photometry
from photutils import Background2D
from photutils import MedianBackground
from scipy.ndimage import shift
import gc                                                           
gc.enable()
import itertools

import warnings
import os
warnings.filterwarnings('ignore')
ROOT_PATH = os.path.normpath(os.getcwd() + os.sep + os.pardir)

#key values for our bands
BANDS = ["V", "B", "R"]

# redefining functions since importing across notebooks seems to be unreliable
def image_stats(image):
    return {
        'Min': np.min(image),
        'Max': np.max(image),
        'Mean': np.mean(image),
        'Mdn': np.median(image),
        'Stdev': np.std(image)
    }

def print_stats(label: str, stats_dict):
    print("\n", label)
    for key, value in stats_dict.items():
        print("\t", key, "\t\t", value)


I (Nico) started with the V-band and viewed the images in ds9 to visually inspect these images for tracking and focussing errors.

V-band images:
Looks good: 3862-3871, 3922, 3923, 3925, 3926, 3927, 4052, 4062, 4064, 4098, 4099\
Slight tracking errors: 3929, 4053, 4063\
Big tracking errors: 3920, 3921, 3924, 3928, 4065\
Lots of cloud: 4058, 4059

Then I looked at the B-band:

Looks good: 3852-3861, 3882, 3885, 3886, 3889, 3890, 3942, 3943, 3946, 3947, 3950, 3951\
Slight focussing errors: 3816-3825\
Slight tracking errors: 3816, 3883, 3887, 3891, 3944, 3948\
Big tracking errors: 3884, 3888, 3945, 3949\

Selecting R-band images - we filter based on qualitative suitability, with a preference for filtering out the images with severe streaking. We can filter the cloud cover images statistically/computationally later on, but a lack of focus/streaking is harder to do.

- 😁 Great : 3872-3881, 4056, 
- 🤨 Okay: 4094, 4095, 3937, 3933, 4057, 
- 😞 Use if must: 3930, 4060, 



Image 4056 here has a relatively elevated background count, but it seems to be the sharpest/the one most in focus and with the least tracking issues. As such, it seems to be the most suitable for shifting as the centroid calculation would be the most reliable.

Ok, we have a reference star. Let's go through the process of shifting based on this star.

In [32]:
# Image numbers that are acceptable for each band

good_images = {
    "B": list(range(3852, 3862)) + [3885, 3886, 3889, 3890, 3942, 3943, 3946, 3947, 3950, 3951],
    "V": list(range(3862, 3872)) + [3922, 3923] + list(range(3925, 3928)) + [4052, 4062, 4064, 4098, 4099],
    "R": list(range(3872, 3882)) + [3933, 3937, 4056, 4057, 4094, 4095]
}

In [40]:
proc_files = {band: ccdproc.ImageFileCollection(
    f"{ROOT_PATH}/src/processed_ims/", glob_include=f"proc_NGC_3293_{band}*") for band in BANDS}

# idx: 20:24 represents the four digits in the file names that represent the file number
# here we're reading only the files that we have selected
scim = {
    band: [CCDData.read(f"{ROOT_PATH}/src/processed_ims/{fn}")
           for fn in image_files.files_filtered(PICTTYPE = 1)
           if int(fn[20:24]) in good_images[band]]
    for band, image_files
    in proc_files.items()}


new_names = {band: [f"s{fn}" for fn in image_files.files_filtered(PICTTYPE=1) if int(fn[20:24]) in good_images[band]] for band, image_files in proc_files.items()}

# number of images we have for each band
print(len(scim["V"]), len(scim["B"]), len(scim["R"]))
print(new_names)

20 20 16
{'V': ['sproc_NGC_3293_V_00003862.fits', 'sproc_NGC_3293_V_00003863.fits', 'sproc_NGC_3293_V_00003864.fits', 'sproc_NGC_3293_V_00003865.fits', 'sproc_NGC_3293_V_00003866.fits', 'sproc_NGC_3293_V_00003867.fits', 'sproc_NGC_3293_V_00003868.fits', 'sproc_NGC_3293_V_00003869.fits', 'sproc_NGC_3293_V_00003870.fits', 'sproc_NGC_3293_V_00003871.fits', 'sproc_NGC_3293_V_00003922.fits', 'sproc_NGC_3293_V_00003923.fits', 'sproc_NGC_3293_V_00003925.fits', 'sproc_NGC_3293_V_00003926.fits', 'sproc_NGC_3293_V_00003927.fits', 'sproc_NGC_3293_V_00004052.fits', 'sproc_NGC_3293_V_00004062.fits', 'sproc_NGC_3293_V_00004064.fits', 'sproc_NGC_3293_V_00004098.fits', 'sproc_NGC_3293_V_00004099.fits'], 'B': ['sproc_NGC_3293_B_00003852.fits', 'sproc_NGC_3293_B_00003853.fits', 'sproc_NGC_3293_B_00003854.fits', 'sproc_NGC_3293_B_00003855.fits', 'sproc_NGC_3293_B_00003856.fits', 'sproc_NGC_3293_B_00003857.fits', 'sproc_NGC_3293_B_00003858.fits', 'sproc_NGC_3293_B_00003859.fits', 'sproc_NGC_3293_B_0000386

We'll also copy these selected images to a new directory so that if we need to access them with ds9, we don't have to go sorting.

In [87]:
import contextlib
with contextlib.suppress(OSError):
    os.mkdir(f"{ROOT_PATH}/src/processed_ims/selected")
with contextlib.suppress(OSError):
    os.mkdir(f"{ROOT_PATH}/src/processed_ims/selected/V")
with contextlib.suppress(OSError):
    os.mkdir(f"{ROOT_PATH}/src/processed_ims/selected/B")
with contextlib.suppress(OSError):
    os.mkdir(f"{ROOT_PATH}/src/processed_ims/selected/R") 

In [90]:
import shutil

temp_names = {band: [fn for fn in image_files.files_filtered(PICTTYPE=1) if int(fn[20:24]) in good_images[band]] for band, image_files in proc_files.items()}

for band, files in temp_names.items():
    for filename in files:
        shutil.copyfile(f"{ROOT_PATH}/src/processed_ims/{filename}", f"{ROOT_PATH}/src/processed_ims/selected/{filename}")


Let's now shift these images. Unfortunately, we have a huge amount of tracking error throughout the night, which means that we need to find a region that contains an isolated bright star.

I used ds9 to animate thoughout the selected images from above, and I found the following star:



In [89]:
# # 4056 is an r band image - get index
# temp_index = new_names["R"].index('sproc_NGC_3293_R_00004056.fits')

# temp = scim["R"][temp_index].copy()
# temp = temp-np.ma.median(temp)

# # Let's use a box from (460, 843) to (540, 923)

# # Determine the centroid - note the python convention for x and y can be flipped!
# # print(temp[843:923, 460:540])
# x1, y1 = centroid_2dg(temp[843:923, 460:540])
# print(x1+460, y1+843)

In [66]:
xoffset = 400
yoffset = 783
xbox = 200
ybox = 200

shift_dict = {band:{} for band in BANDS}

for band, scim_band in scim.items():
    shiftx = []
    shifty = []

    for thisimage in scim_band:
        temp = thisimage.copy()
        temp = temp-np.ma.median(temp)
        x1, y1 = centroid_2dg(temp[yoffset : yoffset + ybox, xoffset : xoffset + xbox])

        print(x1 + xoffset, y1 + yoffset )
        shiftx.append(x1 + xoffset)
        shifty.append(y1 + yoffset)
    
    shift_dict[band]["shiftx"] = shiftx
    shift_dict[band]["shifty"] = shifty

print(shift_dict["B"]["shiftx"])


217.4029544936189 769.4208075971899
574.749162297999 818.7800336386148
322.8128148794853 582.7311741449473
474.707064325264 1133.782797116853
420.41715611246946 711.725169731067
587.3183580094394 831.7583627130213
573.7149678387386 827.0405458072526
525.2692998188915 863.9459115191955
494.098831169713 847.3789165259964
488.7559907080248 840.3948884590638
465.80231318103057 894.5417315453976
462.4075139907107 902.6446424486958
454.4824926077296 906.5529197362575
450.22686616086446 912.3470193853849
439.7663743696954 937.7355354497168
627.2010046492379 803.6106803489469
471.23004470695423 966.4066712535553
564.1873557109939 805.2093654083397
468.9391239388814 914.8249495239553
468.42336646706804 923.1554989348849
450.4542175876531 921.3207439365045
451.32721315641885 922.3907256623761
448.143150457437 921.5713218152521
446.4826962755578 923.0701180873391
371.14034631030245 907.5951755804351
350.72170040008194 969.7876576861756
310.4767528026862 921.4073769585585
331.62637913569404 968.35